In [46]:
import torch
from torch import nn
from torch.utils.data import DataLoader, dataloader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 一、下载训练数据，从开源数据上下载

In [47]:
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

In [48]:
batch_size = 64

# 创建 data_loader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [49]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using Device: {device}")

# define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(True),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

using Device: cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [50]:
# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [51]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss:7f} [{current:>5d}/{size:>5d}]")

In [52]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAverage loss: {(100*correct):>0.1f}%, Avg loss: {(test_loss):>8f}\n")

In [53]:
epochs = 5
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch+1, epochs))
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1/5
Loss: 2.301281 [   64/60000]
Loss: 2.286768 [ 6464/60000]
Loss: 2.273709 [12864/60000]
Loss: 2.270611 [19264/60000]
Loss: 2.250524 [25664/60000]
Loss: 2.221148 [32064/60000]
Loss: 2.224945 [38464/60000]
Loss: 2.188359 [44864/60000]
Loss: 2.180783 [51264/60000]
Loss: 2.152264 [57664/60000]
Test Error: 
Average loss: 45.8%, Avg loss: 2.143187

Epoch 2/5
Loss: 2.153884 [   64/60000]
Loss: 2.135308 [ 6464/60000]
Loss: 2.085690 [12864/60000]
Loss: 2.105088 [19264/60000]
Loss: 2.056637 [25664/60000]
Loss: 1.992226 [32064/60000]
Loss: 2.011758 [38464/60000]
Loss: 1.932681 [44864/60000]
Loss: 1.930069 [51264/60000]
Loss: 1.856909 [57664/60000]
Test Error: 
Average loss: 61.7%, Avg loss: 1.857714

Epoch 3/5
Loss: 1.893032 [   64/60000]
Loss: 1.849621 [ 6464/60000]
Loss: 1.743685 [12864/60000]
Loss: 1.786298 [19264/60000]
Loss: 1.688366 [25664/60000]
Loss: 1.635465 [32064/60000]
Loss: 1.642881 [38464/60000]
Loss: 1.554296 [44864/60000]
Loss: 1.568438 [51264/60000]
Loss: 1.463093 [57664

In [54]:
torch.save(model.state_dict(), "model.pth")
print("Saved model to disk")

Saved model to disk


In [58]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

### Make Predict

In [59]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_dataset[0][0], test_dataset[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
